# Horizontal interpolation: Nearest Neighbours

In [1]:
import nes
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
countries = gpd.read_file('/gpfs/projects/bsc32/models/NES_tutorial_data/gadm_country_mask/gadm_country_ISO3166.shp')

def plot_data(nessy, var_name, zoom=None):
    nessy.create_spatial_bounds()
    lon_bnds, lat_bnds = nessy.get_spatial_bounds_mesh_format()
    
    fig, ax = plt.subplots(1, figsize=(19, 7))
    plt.pcolormesh(lon_bnds, 
                   lat_bnds, 
                   nessy.variables[var_name]['data'].sum(axis=(0, 1)), 
                   cmap='jet', 
                   norm=matplotlib.colors.LogNorm())
    
    countries.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.3)
    plt.colorbar(ax=ax)
    ax.margins(0)
    
    if zoom is not None:
        plt.axis([zoom['x_min'], zoom['x_max'], zoom['y_min'], zoom['y_max']])

    return None

## 1. Read data to interpolate

In [3]:
# Original path: /esarchive/exp/monarch/a4dd/original_files/000/2022111512/MONARCH_d01_2022111512.nc
# Rotated grid from MONARCH
source_path = '/gpfs/projects/bsc32/models/NES_tutorial_data/MONARCH_d01_2022111512.nc'
var_name = 'O3'

In [4]:
source_grid = nes.open_netcdf(path=source_path)
source_grid.keep_vars(var_name)
source_grid.load()

In [ ]:
plot_data(source_grid, var_name)

## 2. Interpolation

### 2.1. Read or Create the destination grid

In [ ]:
# CAMS2_40 Forecast final grid

lat_orig = 30.05
lon_orig = -24.95
inc_lat = 0.1
inc_lon = 0.1
n_lat = 420
n_lon = 700

dst_grid = nes.create_nes(comm=None, info=False, projection='regular',
                          lat_orig=lat_orig, lon_orig=lon_orig, 
                          inc_lat=inc_lat, inc_lon=inc_lon, 
                          n_lat=n_lat, n_lon=n_lon)

### 2.2. Interpolate

#### Nearest

In [ ]:
%time interp_nn1 = source_grid.interpolate_horizontal(dst_grid, kind='NearestNeighbour', n_neighbours=1)

In [ ]:
plot_data(interp_nn1, var_name)

In [ ]:
# Zoom in plot
plot_data(interp_nn1, var_name, zoom={'x_min': -30, 'x_max': 50, 'y_min': 25, 'y_max':75})

#### Bilinear

In [ ]:
%time interp_nn4 = source_grid.interpolate_horizontal(dst_grid, kind='NearestNeighbour', n_neighbours=4)

In [ ]:
plot_data(interp_nn4, var_name, zoom={'x_min': -30, 'x_max': 50, 'y_min': 25, 'y_max':75})

#### Bicubic

In [ ]:
%time interp_nn16 = source_grid.interpolate_horizontal(dst_grid, kind='NearestNeighbour', n_neighbours=16)

In [ ]:
plot_data(interp_nn16, var_name, zoom={'x_min': -30, 'x_max': 50, 'y_min': 25, 'y_max':75})

#### 32 Nearest Neighbours

In [ ]:
%time interp_nn32 = source_grid.interpolate_horizontal(dst_grid, kind='NearestNeighbour', n_neighbours=32)

In [ ]:
plot_data(interp_nn32, var_name, zoom={'x_min': -30, 'x_max': 50, 'y_min': 25, 'y_max':75})